In [ ]:
from keras.layers import *
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop
from keras.activations import relu
from tqdm import tqdm
import numpy as np
import scipy.stats as sc
import matplotlib.pyplot as plt
import pandas as pd
from keras import backend
import numpy as np


from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:

data_1 = np.load('')
data_2 = np.load('')
data_3 = np.load('')



maxim = np.zeros(1000)
minim = np.zeros(1000)
for I in range(954):
  maxim[I] = max(data_1[:,I])
  minim[I] = -min(data_1[:,I])
scale = np.concatenate([maxim,minim])
norm_value_1 = max(scale)
data_1 = data_1 / norm_value_1

maxim = np.zeros(1000)
minim = np.zeros(1000)
for I in range(954):
  maxim[I] = max(data_2[:,I])
  minim[I] = -min(data_2[:,I])
scale = np.concatenate([maxim,minim])
norm_value_2 = max(scale)
data_2 = data_2 / norm_value_2

maxim = np.zeros(1000)
minim = np.zeros(1000)
for I in range(954):
  maxim[I] = max(data_3[:,I])
  minim[I] = -min(data_3[:,I])
scale = np.concatenate([maxim,minim])
norm_value_3 = max(scale)
data_3 = data_3 / norm_value_3


data = np.zeros((954,400,3))

for i in range(954):
  data[i,:,0] = data_1[:,i] 
  data[i,:,1] = data_2[:,i] 
  data[i,:,2] = data_3[:,i] 



In [ ]:
optimiser = RMSprop(0.00005)
ecg_shape = (400,3)
n_critic = 4
clip_value = 0.01


def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)
 

def create_critic():
  critic = Sequential()
  Input_shape = (400,3)
  critic.add(Reshape((400,3),input_shape = (ecg_shape)))
  # Conv 1
  critic.add(Conv1D(64, kernel_size=5, strides=1,input_shape=(ecg_shape)))
  critic.add(MaxPooling1D(pool_size =5, strides = 1))
  critic.add(LeakyReLU(0.2))

  # Conv 2
  critic.add(Conv1D(128, kernel_size=5, strides=1))
  critic.add(MaxPooling1D(pool_size = 5, strides = 1))
  critic.add(LeakyReLU(0.2))

  # Conv 3
  critic.add(Conv1D(256, kernel_size=5, strides=1))
  critic.add(MaxPooling1D(pool_size = 5, strides = 2))
  critic.add(LeakyReLU(0.2))

  # Conv 4
  critic.add(Conv1D(512, kernel_size=5, strides=1))
  critic.add(MaxPooling1D(pool_size = 5, strides = 2))
  critic.add(LeakyReLU(0.2))


  critic.add(Flatten())
  
  critic.add(Dense(units=3, activation='linear'))

  critic.compile(loss=wasserstein_loss, metrics=['accuracy'], optimizer=optimiser)
  critic.summary()

  return critic

def create_generator():

  Input_shape = (400,1)
  noise = Input(shape=Input_shape)
  x = Reshape((400,1),input_shape = (400,1))
  x = Bidirectional(LSTM(128, return_sequences=True))(noise)

  x1 = Bidirectional(LSTM(128, return_sequences=True))(x)
  x1 = Dropout(0.1)(x1)
  x1 = Flatten()(x1)
  # x1 = Dense(ecg_shape[0], activation='tanh')(x1)
  x1 = tf.expand_dims(x1,axis=2)


  x2 = Bidirectional(LSTM(128, return_sequences=True))(x)
  x2 = Dropout(0.1)(x2)
  x2 = Flatten()(x2)
  # x2 = Dense(ecg_shape[0], activation='tanh')(x2)
  x2 = tf.expand_dims(x2,axis=2)

  x3 = Bidirectional(LSTM(128, return_sequences=True))(x)
  x3 = Dropout(0.1)(x3)
  x3 = Flatten()(x3)
  # x3 = Dense(ecg_shape[0], activation='tanh')(x3)
  x3 = tf.expand_dims(x3,axis=2)

  
  x_out = Concatenate(axis=2)([x1,x2,x3])
  x_out = Permute((2,1))(x_out)
  x_out = Dense(400, activation = 'tanh')(x_out)
  x_out = Permute((2,1))(x_out) 
  
  generator = Model(noise,x_out)
  generator.summary()

  return generator


def create_GAN(critic, generator):
  critic.trainable = False

  GAN = Sequential()
  GAN.add(Input(shape=(400,1)))
  GAN.add(generator)
  GAN.add(critic)

  GAN.compile(loss=wasserstein_loss, optimizer=optimiser)
  GAN.summary()
  return GAN

In [ ]:
import tensorflow as tf

def compute_kernel(x, y):
    x_size = tf.shape(x)[0]
    y_size = tf.shape(y)[0]
    dim = tf.shape(x)[1]
    tiled_x = tf.tile(tf.reshape(x, tf.stack([x_size, 1, dim])), tf.stack([1, y_size, 1]))
    tiled_y = tf.tile(tf.reshape(y, tf.stack([1, y_size, dim])), tf.stack([x_size, 1, 1]))
    return tf.exp(-tf.reduce_mean(tf.square(tiled_x - tiled_y), axis=2) / tf.cast(dim, tf.float64))

def compute_mmd(x, y, sigma_sqr=1.0):
    x_kernel = compute_kernel(x, x)
    y_kernel = compute_kernel(y, y)
    xy_kernel = compute_kernel(x, y)
    return tf.reduce_mean(x_kernel) + tf.reduce_mean(y_kernel) - 2 * tf.reduce_mean(xy_kernel)

In [ ]:

def mmd_loss_1():

  real_data_set =  np.load('')
  idx = np.random.randint(0, 954, 800)
  real_data = real_data_set[:,idx]

  #print(real_data.shape)

  fake_data = np.zeros((400,800))
  data = np.zeros(400)
  for i in range(800):
    noise = tf.random.normal([400,1])
    #print(noise.shape)
    noise1 = tf.expand_dims(noise, axis=0)
    #print(noise1.shape)
    ecgs = generator.predict(noise1)
    denorm_ecgs_1 = ecgs[0,:,0] * norm_value_1
    #print(denorm_ecgs_1.shape)

    for j in range(400):
      data[j] = denorm_ecgs_1[j]
    fake_data[:,i] = data

  real_data = np.transpose(real_data)
  fake_data = np.transpose(fake_data)

  mmd_loss_1 = compute_mmd(real_data,fake_data)
  print(mmd_loss_1)

  return mmd_loss_1


def mmd_loss_2():

  real_data_set =  np.load('')
  idx = np.random.randint(0, 954, 800)
  real_data = real_data_set[:,idx]

  #print(real_data.shape)

  fake_data = np.zeros((400,800))
  data = np.zeros(400)
  for i in range(800):
    noise = tf.random.normal([400,1])
    #print(noise.shape)
    noise1 = tf.expand_dims(noise, axis=0)
    #print(noise1.shape)
    ecgs = generator.predict(noise1)
    denorm_ecgs_2 = ecgs[0,:,1] * norm_value_2

    for j in range(400):
      data[j] = denorm_ecgs_2[j]
    fake_data[:,i] = data


  real_data = np.transpose(real_data)
  fake_data = np.transpose(fake_data)

  mmd_loss_2 = compute_mmd(real_data,fake_data)
  print(mmd_loss_2)

  return mmd_loss_2


def mmd_loss_3():

  real_data_set =  np.load('')
  idx = np.random.randint(0, 954, 800)
  real_data = real_data_set[:,idx]

  #print(real_data.shape)

  fake_data = np.zeros((400,800))
  data = np.zeros(400)
  for i in range(800):
    noise = tf.random.normal([400,1])
    #print(noise.shape)
    noise1 = tf.expand_dims(noise, axis=0)
    #print(noise1.shape)
    ecgs = generator.predict(noise1)
    denorm_ecgs_3 = ecgs[0,:,2] * norm_value_3

    for j in range(400):
      data[j] = denorm_ecgs_3[j]
    fake_data[:,i] = data


  real_data = np.transpose(real_data)
  fake_data = np.transpose(fake_data)

  mmd_loss_3 = compute_mmd(real_data,fake_data)
  print(mmd_loss_3)

  return mmd_loss_3

In [ ]:
def show_results(generator,norm_value_1,norm_value_2,norm_value_3):
  noise = tf.random.normal([400,1])
  noise1 = tf.expand_dims(noise, axis=0)
  ecgs = generator.predict(noise1)

  denorm_ecgs_1 = ecgs[0,:,0] * norm_value_1
  denorm_ecgs_2 = ecgs[0,:,1] * norm_value_2
  denorm_ecgs_3 = ecgs[0,:,2] * norm_value_3

  fig, (ax1, ax2,ax3) = plt.subplots(1,3)

  fig.set_figheight(4)
  fig.set_figwidth(15)

  ax1.plot(denorm_ecgs_1)
  ax1.set(xlabel='Timestep', ylabel='Amplitude(mV)')
  ax1.set_xlim((0,400))
  ax1.set_title('Lead I')
  ax1.grid()
  ax2.plot(denorm_ecgs_2)
  ax2.set(xlabel='Timestep', ylabel='Amplitude(mV)')
  ax2.set_xlim((0,400))
  ax2.set_title('Lead II')
  ax2.grid()
  ax3.plot(denorm_ecgs_3)
  ax3.set(xlabel='Timestep', ylabel='Amplitude(mV)')
  ax3.set_xlim((0,400))
  ax3.set_title('Lead III')
  ax3.grid()
  
  fig.tight_layout()


In [ ]:
import tensorflow as tf

batch_size = 64
epochs = 2000
critic = create_critic()
generator = create_generator()
GAN = create_GAN(critic, generator)


noise_array = np.array(tf.random.normal([1000,400,1]))
print('BREAK')
print('noise_array.shape')
print(noise_array.shape)
print('BREAK')
d_losses, d_accuracy, g_losses = [], [], []

for epoch in tqdm(range(epochs)):
  
  for _ in range(n_critic):
    
    idx = np.random.randint(0, high = 954, size= batch_size)
    real_ecgs = data[idx]


    idx = np.random.randint(0,high= 1000 ,size=batch_size)
    noise = noise_array[idx,:,:]

    fake_ecgs = generator.predict(noise,batch_size=batch_size)

    X_real = real_ecgs
    X_fake = fake_ecgs

    y_real = -np.ones((batch_size,3)) 
    y_fake = np.ones((batch_size,3)) 

    critic.trainable = True
    c_loss_real = critic.train_on_batch(X_real ,y_real)
    c_loss_fake = critic.train_on_batch(X_fake ,y_fake)
    critic.trainable = False

    for l in critic.layers:
        weights = l.get_weights()
        weights = [np.clip(w, -clip_value, clip_value) for w in weights]
        l.set_weights(weights)

  y = - np.ones((batch_size,3))
  g_loss = GAN.train_on_batch(noise, y)

  
  if epoch%50 == 0:
    show_results(generator,norm_value_1,norm_value_2,norm_value_3)
    show_results(generator,norm_value_1,norm_value_2,norm_value_3)
    show_results(generator,norm_value_1,norm_value_2,norm_value_3)
    show_results(generator,norm_value_1,norm_value_2,norm_value_3)
    show_results(generator,norm_value_1,norm_value_2,norm_value_3)
    show_results(generator,norm_value_1,norm_value_2,norm_value_3)
    show_results(generator,norm_value_1,norm_value_2,norm_value_3)
   
    
  

    

  
    # mmd_loss_1()
    # mmd_loss_2()
    # mmd_loss_3()

plt.figure
plt.plot(d_losses, 'r')
plt.plot(g_losses, 'b')
plt.show()

plt.plot(d_accuracy)
plt.show()



In [ ]:
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)
show_results(generator,norm_value_1,norm_value_2,norm_value_3)